# Workshop on sensor processing using machine learning
Course: NICF- Intelligent Sensing and Sense Making (SF)

Website: https://www.iss.nus.edu.sg/executive-education/course/detail/intelligent-sensing-and-sense-making/analytics-and-intelligent-systems

Contact: Tian Jing

Email: tianjing@nus.edu.sg

## Objective
In this workshop, we will perform following three tasks:

- Introduction to autoencoder model and Keras
- Perform anomaly detection using autoencoder as reconstruction model


## Installation guideline (local PC)
- Open `Anaconda Prompt`

- Append the channel `conda-forge` into your conda configuration.

`conda config --append channels conda-forge`

- Create a new virtual environment `issmv` or install additional packages in your own environment

[Windows, CPU version]

`conda create -n issmv python=3.6 numpy=1.15.1 matplotlib=2.2.3 tensorflow=1.10.0 scipy=1.1.0 scikit-learn=0.19.1 spyder=3.3.2 yaml=0.1.7 keras=2.2.4 notebook=5.7.4 pandas=0.24.2 pywavelets=1.0.2`

[Windows, GPU version, CUDA 9.0]

`conda create -n issmv python=3.6 numpy=1.15.1 matplotlib=2.2.3 tensorflow-gpu=1.12.0 scipy=1.1.0 scikit-learn=0.19.1 spyder=3.3.2 yaml=0.1.7 keras-gpu=2.2.4 notebook=5.7.4 pandas=0.24.2 pywavelets=1.0.2`

- Activate the environment `issmv`

`conda activate issmv`

- Browse to the folder that contains the workshop files, then run `jupyter notebook`

`jupyter notebook`

## Submission guideline
Once you finish the workshop, rename your .ipynb file to your name, and submit your .ipynb file into LumiNUS.


In [1]:
# Check GPU setup in Colab
%tensorflow_version 1.x 
import tensorflow as tf
# Silence the tensorflow warning message
tf.get_logger().setLevel('ERROR')

print("Tensorflow version: ", tf.__version__)
print(tf.test.gpu_device_name())
# Your expected output will be '/device:GPU:0'

TensorFlow 1.x selected.
Tensorflow version:  1.15.2
/device:GPU:0


In [2]:
import os
import numpy as np
import keras
import pickle
import pandas as pd
from tensorflow.keras.utils import plot_model
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Input
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler


Using TensorFlow backend.


In [3]:
# Then you’ll see a link, click on that link, allow access
# Copy the code that pops up, paste it in the box, hit enter

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
# Change working directory to be current folder
# os.chdir('/content/gdrive/My Drive/Your Folder Name/Your sub Folder Name')
os.chdir('/content/gdrive/My Drive/ISSM/day3')
!ls


data
wk_abnormal_colab_v202007.ipynb
wk_abnormal_colab_v202007_solution.ipynb
wk_abnormal_colab_v202009TT.ipynb
wk_abnormal_colab_v202009TT_solution.ipynb
wk_abnormal_colab_v4.0.ipynb


# Exercise: Machine health monitoring

- Reference: R. Zhao, R. Yan, Z. Chen, K. Mao, P. Wang, R. Gao, Deep learning and its applications to machine health monitoring, Mechanical Systems and Signal Processing, Vol. 115, Jan. 2019, pp. 213-237.

- pdf paper, https://arxiv.org/abs/1612.07640
- code, https://github.com/ClockworkBunny/MHMS_DEEPLEARNING

In [5]:
# Configuration
encoder_dimension = 10
NUM_EPOCH = 10
BATH_SIZE = 24

In [6]:
# Load data 
x_train_pd = pd.read_csv("data/machine_sensor_train.csv",header=None)
x_train = x_train_pd.values
x_test_pd = pd.read_csv("data/machine_sensor_test.csv",header=None)
x_test = x_test_pd.values

print('Train shape: ', x_train.shape, 'Test shape:', x_test.shape)

Train shape:  (600, 70) Test shape: (300, 70)


In [7]:
myModel = Sequential()
myModel.add(Dense(encoder_dimension, activation='relu', input_shape=(x_train.shape[1], )))
myModel.add(Dense(encoder_dimension, activation='relu'))
myModel.add(Dense(x_train.shape[1])) # note: this is regression model
myModel.compile(optimizer='adam', loss='mse')
myModel.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                710       
_________________________________________________________________
dense_1 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_2 (Dense)              (None, 70)                770       
Total params: 1,590
Trainable params: 1,590
Non-trainable params: 0
_________________________________________________________________


In [8]:
# Train the model
myModel.fit(x_train, x_train, epochs=NUM_EPOCH, batch_size=BATH_SIZE, shuffle=True)

Train on 600 samples
Epoch 1/10
600/600 [==============================] - 8s 13ms/sample - loss: 0.4125
Epoch 2/10
600/600 [==============================] - 0s 76us/sample - loss: 0.2959
Epoch 3/10
600/600 [==============================] - 0s 79us/sample - loss: 0.1828
Epoch 4/10
600/600 [==============================] - 0s 74us/sample - loss: 0.1098
Epoch 5/10
600/600 [==============================] - 0s 80us/sample - loss: 0.0747
Epoch 6/10
600/600 [==============================] - 0s 81us/sample - loss: 0.0590
Epoch 7/10
600/600 [==============================] - 0s 88us/sample - loss: 0.0503
Epoch 8/10
600/600 [==============================] - 0s 83us/sample - loss: 0.0436
Epoch 9/10
600/600 [==============================] - 0s 77us/sample - loss: 0.0372
Epoch 10/10
600/600 [==============================] - 0s 85us/sample - loss: 0.0319


In [9]:
sample_index = 0
thre = 10 # A manually-defined threshold

# Step 1: Select the test sample (such as the 0-th sample)
test_data_orig = x_test[sample_index:sample_index+1,:]
# print(test_data_orig.shape)

# Step 2: Perform prediction using the trained autoencoder model
test_data_predict = myModel.predict(test_data_orig)

# Step 3: Calculate the reconstruction error and make decision on anomaly detection
dist = np.linalg.norm(test_data_orig - test_data_predict, axis=-1)

if (dist>thre):
    print('Anomaly detected: the distance is %.2f' % dist)
else:
    print('Normal: the distance is %.2f' % dist)
    


Normal: the distance is 1.74


$\color{red}{\text{Q1. Complete the code to build autoencoder model for abnormal machine sensory signal detection.}}$

Tasks

- Peform the test on x_test and determine the test data (row index) with the LARGEST reconstruction error based on your trained model
- Plot the data that is found by your model

In [10]:
# Provide your solution to Q1 here

$\color{red}{\text{Q2. Identify one use case of anomaly detection and describe your use case.}}$

In [11]:
# Provide your solution to Q2 here. (no need programming)

**Once you finish the workshop, rename your .ipynb file to your name, and submit your .ipynb file into LumiNUS.**

Have a nice day!